In [1]:
#install pypianoroll, if first time running notebook
#%pip install pypianoroll


Note: you may need to restart the kernel to use updated packages.


In [1]:
#import libraries

import numpy as np
import pandas as pd
import pypianoroll

In [25]:
#path to our folder with midi files
#filepath = "/Users/annachesson/code/ai_saturdays/google_collab/aisaturdays_collab/arabic_tribal_rhythms/waaHida_02.mid"

# transforming to array, use pretty_midi? 
https://towardsdatascience.com/generate-piano-instrumental-music-by-using-deep-learning-80ac35cdbd2e


to_pretty_midi(constant_tempo=None, constant_velocity=100)[source]
Convert to a pretty_midi.PrettyMIDI instance.

Notes

Only constant tempo is supported by now.

The velocities of the converted pianorolls are clipped to [0, 127], i.e. values below 0 and values beyond 127 are replaced by 127 and 0, respectively.

Adjacent nonzero values of the same pitch will be considered a single note with their mean as its velocity.

Parameters
constant_tempo (int) – The constant tempo value of the output object. Defaults to use the first element of tempo.

constant_velocity (int) – The constant velocity to be assigned to binarized tracks. Defaults to 100.

Returns
pm – The converted pretty_midi.PrettyMIDI instance.

In [26]:
#import pretty_midi library, pip install pretty_midi
import pretty_midi

#straight from tutorial. here we convert a midifile to pretty_midi directly, take the piano channel, then get the binary array

midi_pretty_format = pretty_midi.PrettyMIDI(filepath)
piano_midi = midi_pretty_format.instruments[0] # Get the piano channels
piano_roll = piano_midi.get_piano_roll()
whole_roll = midi_pretty_format.get_piano_roll() #fs = fs

In [5]:
whole_roll

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

## What is fs? ->frames per second
After we get the array of piano roll, we convert them into dictionary. The dictionary will start from the time where the note is played. For example, in the picture above, we start from 28 (If we convert to second, assume we convert to piano_roll at 5 fps, the music start playing its notes at 5.6 s which we can get by 28 divided by 5).

# Questions so far

1. Is there a difference, or is it necessary, to go from midi->pypiano->multitrack object->pretty_midi->binary array? Or can we just read the midi file directly with pretty_midi (midi->pretty_midi->binary array). -Update: in the end I have decided to read directly with the pretty_midi library as done in the tutorial

2. the tutorial takes only the piano channel (midi_pretty_format.instruments[0]). Presumably we want to take all the instrument channels in our own training data set

3. What really is frames per second? why does it matter?

In [27]:
#when you import the notebook it runs the some library installations necessary for it to run
import repository

In [28]:
#unzips the folder
import zipfile
with zipfile.ZipFile("arabic_tribal_rhythms_ZIP.zip","r") as zip_ref:
    zip_ref.extractall()#where you want the file to be unzipped, I want it in the same folder?



'/Users/annachesson/code/ai_saturdays/google_collab/aisaturdays_collab/arabic_tribal_rhythms/arabic_tribal_rhythms_ZIP'

In [47]:
#name of folder: 'maestro-v1.0.0/**/*.midi'


def get_list_midi(folder = 'arabic_tribal_rhythms/arabic_tribal_rhythms/*.mid', seed_int = 666):
  """Get the list of all midi file in the folders
  
  Parameters
  ==========
  folder : str
    The midi folder.
  seed_int : int
    the random seed.
  
  Returns
  =======
  The midi files
  
  """
  list_all_midi = glob.glob(folder)
  seed(seed_int)
  shuffle(list_all_midi)
  return list_all_midi

list_all_midi = get_list_midi()

In [48]:
len(list_all_midi)

98

# The following function from notetokenizer.py creates a dictionary of notes from the pianoroll array

def generate_dict_time_notes(list_all_midi, batch_song = 16, start_index=0, fs=30, use_tqdm=True):
    """ Generate map (dictionary) of music ( in index ) to piano_roll (in np.array)

    Parameters
    ==========
    list_all_midi : list
        List of midi files
    batch_music : int
      A number of music in one batch
    start_index : int
      The start index to be batched in list_all_midi
    fs : int
      Sampling frequency of the columns, i.e. each column is spaced apart
        by ``1./fs`` seconds.
    use_tqdm : bool
      Whether to use tqdm or not in the function

    Returns
    =======
    dictionary of music to piano_roll (in np.array)

In [49]:
from repository import generate_dict_time_notes, generate_input_and_target, process_notes_in_song, generate_batch_song, NoteTokenizer

In [44]:
from time import sleep

In [45]:
#basically all this tqdm does in the end is display these progress bars.... fml
from tqdm import notebook as tqdm
with tqdm.tqdm(range(1)) as pbar:
  pass

In [50]:
dictionary = generate_dict_time_notes(list_all_midi, batch_song = 16, start_index=0, fs=30, use_tqdm=True)

In [51]:
# # Sample 100 midi files from the datasets
# These midi files will be used to train the neural network

# In[6]:


# Get 200 midis file from the datasets.. 

sampled_200_midi = list_all_midi[0:100]  


# We create a map of note -> index here using NoteTokenizer that we've defined before.  
# This object will be used to transform the list of notes to be ready for the input of Neural Network

# In[7]:


batch = 1
start_index = 0
note_tokenizer = NoteTokenizer()

for i in tqdm.tqdm(range(len(sampled_200_midi))):
    dict_time_notes = generate_dict_time_notes(sampled_200_midi, batch_song=1, start_index=i, use_tqdm=False, fs=5)
    full_notes = process_notes_in_song(dict_time_notes)
    for note in full_notes:
        note_tokenizer.partial_fit(list(note.values()))
   


# In[8]:


note_tokenizer.add_new_note('e') # Add empty notes


# In[9]:


unique_notes = note_tokenizer.unique_word
print(unique_notes)



1
